In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [2]:
import numpy as np

In [633]:
train_dataset = dsets.MNIST(root='./MNIST/', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./MNIST/', 
                           train=False, 
                           transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=128,
                                           shuffle=True)

In [666]:
class ActNorm(nn.Module):
    def __init__(self, h, w, c):
        super().__init__()
        self.shape = (h, w, c)
        self.initialized = False
        self.weights = nn.Parameter(torch.Tensor(c))
        self.bias = nn.Parameter(torch.Tensor(c))
#         self.weights.data.uniform_(-0.1, 0.1)
#         self.bias.data.uniform_(-0.1, 0.1)
        
    def forward(self, inp, logdet):
        if not self.initialized:
            c = self.shape[-1]
            self.weights.data = 1/inp.transpose(0, 1).contiguous().view(c, -1).std(1)
            self.bias.data = -(inp * self.weights[..., None, None]).transpose(0, 1).contiguous().view(c, -1).mean(1)
            self.initialized = True
        
        return inp * self.weights[..., None, None] + self.bias[..., None, None], logdet + self.shape[0] * self.shape[1] * torch.sum(torch.abs(self.weights))
    
    def reverse(self, out):
        return (out - self.bias) / self.weights

class InvertibleConv(nn.Module):
    def __init__(self, h, w, c):
        super().__init__()
        self.shape = (h, w, c)
        self.weight = nn.Parameter(torch.from_numpy(np.linalg.qr(np.random.randn(c, c))[0]).float())
                                    
    def forward(self, inp, logdet):
        return torch.einsum("abcd,eb->aecd", (inp, self.weight)), logdet + self.shape[0] * self.shape[1] * torch.log(torch.abs(torch.det(self.weight)))
    
    def reverse(self, out):
        return torch.einsum("abcd,eb->aecd", (inp, torch.inverse(self.weight)))

class AffineCoupling(nn.Module):
    def __init__(self, h, w, c):
        super().__init__()
        self.NN = nn.Sequential(
            nn.Conv2d(c//2, c//2, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(c//2, c, 3, padding=1),
        )
        
    def forward(self, inp, logdet):        
        x_a, x_b = torch.split(inp, inp.shape[1] // 2, dim=1)
        log_s, t = torch.split(self.NN(x_b), inp.shape[1]//2, 1)
        s = torch.exp(log_s)
        y_a = s * x_a + t
        return torch.cat([y_a, x_b], dim = 1), logdet + torch.sum(torch.log(torch.abs(s)))
    
    def reverse(self, out):
        y_a, y_b = torch.split(out, out.shape[1] // 2, dim = 1)
        log_s, t = torch.split(self.NN(y_b), inp.shape[1]//2, 1)
        s = torch.exp(log_s)
        x_a = (y_a - t) / s
        return torch.cat([x_a, y_b], dim = 1)
    
class Flow(nn.Module):
    def __init__(self, K, h, w, c):
        super().__init__()
        self.k = K
        self.actnorms = nn.ModuleList(ActNorm(h, w, c) for i in range(K))
        self.invconvs = nn.ModuleList(InvertibleConv(h, w, c) for i in range(K))
        self.couplings = nn.ModuleList(AffineCoupling(h, w, c) for i in range(K))
        
    def forward(self, inp, logdet, z):        
        for i in range(self.k):
            out, logdet = self.actnorms[i](inp, logdet)
            out, logdet = self.invconvs[i](out, logdet)
            out, logdet = self.couplings[i](out, logdet)
        
        return out, logdet, z
    
    def reverse(self, out):
        out = self.actnorm.reverse(out)
        out = self.invconv.reverse(out)
        out = self.coupling.reverse(out)
        return out

In [667]:
def pixel_unshuffle(input_, block_size=2):
    b, c, h, w = input_.shape

    assert h % block_size == 0 and w % block_size == 0,\
        f"Shape must be divisible by block_size, got {input_.shape}"

    oc = c * block_size * block_size;
    oh = h // block_size;
    ow = w // block_size;

    input_reshaped = input_.view(b, c, oh, block_size, ow, block_size)
    return input_reshaped.permute(0, 1, 3, 5, 2, 4).reshape(b, oc, oh, ow)
    
class Squeeze(nn.Module):
    def __init__(self, block_size=2):
        super().__init__()
        self.block_size = block_size 

    def forward(self, input_, logdet, z):
        return pixel_unshuffle(input_, self.block_size), logdet, z
    
    def reverse(self, input_, z):
        return F.pixel_shuffle(input_, self.block_size), z

In [668]:
class Split(nn.Module):
    def forward(self, input_, logdet, z):
        out, zi = torch.split(input_, input_.shape[1]//2, 1)

        return out, logdet, z + [zi]
    
    def reverse(self, input_, z):
        out = torch.cat([input_, z[-1]], 1)
        
        return out, z[:-1]

In [674]:
class LogLikelihood(nn.Module):
    def forward(self, input_, logdet, z):
        return input_, logdet + torch.distributions.normal.Normal.log_prob(torch.cat([z_.contiguous().view(-1) for z_ in z])), z

In [675]:
class GlowModel(nn.Module):
    def __init__(self, h, w, c, K, L):
        super().__init__()
        
        self.layers = nn.ModuleList()
        
        for i in range(1, L):
            self.layers.append(Squeeze())
            self.layers.append(Flow(K, h//(2**i), w//(2**i), c*2**(i+1)))
            self.layers.append(Split())

        i += 1
        self.layers.append(Squeeze())
        self.layers.append(Flow(K, h//(2**i), w//(2**i), c*2**(i+1)))
        self.layers.append(LogLikelihood())
        self.k = K
        self.l = L
        
    def forward(self, input_, logdet, z):
        for layer in self.layers:
            input_, logdet, z = layer(input_, logdet, z)
            
        return input_, logdet, z
    
    def reverse(self, input_, z):
        for layer in self.layers[::-1]:
            input_, z = layer.reverse(input_, z)

        return input_, z

In [676]:
model = GlowModel(28, 28, 1, 8, 2).to(device)

In [677]:
from tqdm import tqdm_notebook as tqdm

def train_epoch(model, optimizer, batchsize=32):
    loss_log, acc_log = [], []
    model.train()
    for x_batch, _ in tqdm(train_loader):
        data = x_batch.to(device)
        optimizer.zero_grad()
        out, logdet, z = model(data, 0, [])

        loss = -logdet
        loss.backward()
        optimizer.step()
        loss = loss.item()
        loss_log.append(loss)
    return loss_log

def plot_history(train_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    plt.plot(train_history, label='train', zorder=1)
    
#     points = np.array(val_history)
#     plt.scatter(points[:, 0], points[:, 1], marker='+', s=180, c='orange', label='val', zorder=2)
    plt.xlabel('train steps')
    
    plt.legend(loc='best')
    plt.grid()

    plt.show()
    
def train(model, opt, n_epochs):
    train_log, train_acc_log = [], []
    val_log, val_acc_log = [], []

    batchsize = 32

    for epoch in range(n_epochs):
        train_loss = train_epoch(model, opt, batchsize=batchsize)
        train_log.extend(train_loss)

        steps = train_dataset.train_labels.shape[0] / batch_size

        clear_output()
        plot_history(train_log)    
            
    print("Final error: {:.2%}".format(1 - val_acc_log[-1][1]))

In [678]:
train(model, optim.Adam(model.parameters()), 10)

TypeError: log_prob() missing 1 required positional argument: 'value'